In [ ]:
query = "select * from emart-datafabric.samples.data_usage"
path = "tmp/test_20200825"

In [ ]:
bucket = "mkt-insight-datahub"
destination = f"s3-prd://{bucket}/{path}/"

In [ ]:
from pydatafabric.gcp import bq_to_df

df = bq_to_df(query)
df.write.mode("overwrite").option("compression", "gzip").csv(destination)